In [3]:
import pandas as pd
import numpy as np
import re
import pandas as pd
from googletrans import Translator

# Scammer profiles

In [2]:
scammers_df = pd.read_csv('../data/merged_dataset.csv')
scammers_df.replace("–", np.nan, inplace=True)
scammers_df.head()

,username,name,age,location,ethnicity,occupation,status,phone,inet,email,description,messages,justifications
0,fredJ,Fred Millestone,49,"San Diego, California, United States",white,military,NaN,NaN,178.238.213.161,fred.millestone@yahoo.com,"I am a simple and easy going man,I do not like...",hello beauty can we chat? add me fred.millesto...,IP is a proxy\nIP doesn’t correspond location\...
1,donald200,Donald Steve,46,Ireland,white,Self Employed,widowed,widowed,164.82.146.3,donaldsteve200@yahoo.com,"Am smart, organized, intelligent, honest, cari...","How are you doing,I’m Donald from Ireland… am ...",IP is a proxy\nIP doesn’t correspond location\...
2,huddleman,Peter,45,"Arlington, or Andrews, or Dallas, Texas, Unite...",white,military,widowed,widowed,69.115.19.232,frankiess8928@gmail.com,"I am sweet, thoughtful, kind, intelligent, and...",NaN,IP is a proxy\nIP doesn’t correspond location\...
3,jannycutelove,Jane Douglas,33,"Vancouver, Canada",white,student,single,single,67.191.26.153,janedouglas231@yahoo.com,"I am an attractive, physically and mentally he...",NaN,IP is a proxy\nIP doesn’t correspond location\...
4,Richardholdy,David Holdridge,47,"Bedford, Pennsylvania, United states",white,military,widowed,widowed,199.15.250.244,david_holdy@yahoo.com,"I am a loyal,friendly ,and always supportive ....",Its really a pleasure to hear from back from u...,IP is a proxy\nIP doesn’t correspond location\...


In [3]:
scammers_df = scammers_df.drop(["username", "name", "phone", "inet", "email", "messages", "justifications"], axis=1)

In [4]:
scammers_df = scammers_df.dropna().reset_index(drop=True)

In [5]:
scammers_df

,age,location,ethnicity,occupation,status,description
0,46,Ireland,white,Self Employed,widowed,"Am smart, organized, intelligent, honest, cari..."
1,45,"Arlington, or Andrews, or Dallas, Texas, Unite...",white,military,widowed,"I am sweet, thoughtful, kind, intelligent, and..."
2,33,"Vancouver, Canada",white,student,single,"I am an attractive, physically and mentally he..."
3,47,"Bedford, Pennsylvania, United states",white,military,widowed,"I am a loyal,friendly ,and always supportive ...."
4,32,"Toronto, Canada",white,nurse,single,My goal is a monogamous relationship leading t...
...,...,...,...,...,...,...
3241,49,"Dallas, Texas, United States",native american,engineer,widowed,"I am a straight forward and intelligent man, I..."
3242,59,"Texas City, Texas, United States, or Washingto...",white,military,widowed,Am down to earth and God fearing man who will ...
3243,54 or 60,"Ashburn, Virginia, or Florida, Perry, United S...",white,Architect,widowed,"I am a kind, thoughtful and friendly person. I..."
3244,24,"Hamburg, Germany",black,student,single,"My Name is Jovita, I am a young good looking g..."


In [6]:
statuses = np.asarray(scammers_df["status"].value_counts().index)
idx_to_remove = np.where(np.asarray(scammers_df["status"].value_counts().where(lambda x : x == 1, 0)))[0]
status_to_remove = statuses[idx_to_remove]

In [7]:
status_to_remove

array(['seprated', 'widowed or divorced', 'married or divorced',
       'divorced or single', 'singlescammer’s real name: Olusegun',
       'married (and looking for marriage)', 'Widower', 'military',
       'Single', 'divorce', 'Civil Engineer', 'nurse', 'windoew',
       'Nursing assistant'], dtype=object)

In [8]:
scammers_df = scammers_df[~scammers_df["status"].isin(status_to_remove)]
scammers_df

,age,location,ethnicity,occupation,status,description
0,46,Ireland,white,Self Employed,widowed,"Am smart, organized, intelligent, honest, cari..."
1,45,"Arlington, or Andrews, or Dallas, Texas, Unite...",white,military,widowed,"I am sweet, thoughtful, kind, intelligent, and..."
2,33,"Vancouver, Canada",white,student,single,"I am an attractive, physically and mentally he..."
3,47,"Bedford, Pennsylvania, United states",white,military,widowed,"I am a loyal,friendly ,and always supportive ...."
4,32,"Toronto, Canada",white,nurse,single,My goal is a monogamous relationship leading t...
...,...,...,...,...,...,...
3241,49,"Dallas, Texas, United States",native american,engineer,widowed,"I am a straight forward and intelligent man, I..."
3242,59,"Texas City, Texas, United States, or Washingto...",white,military,widowed,Am down to earth and God fearing man who will ...
3243,54 or 60,"Ashburn, Virginia, or Florida, Perry, United S...",white,Architect,widowed,"I am a kind, thoughtful and friendly person. I..."
3244,24,"Hamburg, Germany",black,student,single,"My Name is Jovita, I am a young good looking g..."


In [9]:
scammers_df = scammers_df[scammers_df["status"] != "single or divorced"]

In [10]:
scammers_df.loc[:, "status"] = scammers_df["status"].apply(lambda x : x.lower())

In [11]:
scammers_df["status"].value_counts()

status
single       1692
widowed       988
divorced      513
separated      22
widower         9
widow           4
married         2
Name: count, dtype: int64

In [12]:
scammers_df.loc[:, "status"] = scammers_df["status"].apply(lambda x : "widowed" if x == "widower" else x)
scammers_df.loc[:, "status"] = scammers_df["status"].apply(lambda x : "widowed" if x == "widow" else x)

In [21]:
scammers_df = pd.read_csv('../data/output_scam_profiles.csv')

In [22]:
# drop rows in scammers_df if "age" column contains more than just numbers
scammers_df = scammers_df[scammers_df["age"].apply(lambda x : re.match("^\d+$", str(x)) is not None)]
scammers_df

,age,location,ethnicity,occupation,status,description,scam
0,54,"Washingtom DC, United States, or Kabul, Afghan...",native american,medical,widowed,"Am an easy going type of woman, very resourcef...",1
1,63,"Ohio, or Colorado, or Arkansas, United States",white,engineering,widowed,"I am independent man who has my own busy life,...",1
2,47,"Los Angleles, California, United States",native american,engineering,divorced,"Hey mates, I am Rob i am looking for a serious...",1
3,49,"Rockville, Maryland, United States, or Kabul, ...",white,government,widowed,"Hello,i am here to look for a good relationshi...",1
4,42,"London, United Kingdom, or Helsinki, Finland",white,self-employed,divorced,"a nice cool, gentle, honest and funning person...",1
...,...,...,...,...,...,...,...
3235,34,"Donetsk, Ukraine",white,sales,single,"I am – this is a simple soul, which difficult ...",1
3236,55,"Texas City, United States",white,contractor,single,looking to find love,1
3237,45,"Fort Lauderdale, Florida, United States",native american,military,single,Hello am Ron Riley Jack and am new here i am l...,1
3238,44,"London, United Kingdom",white,other,widowed,"I am Honest Man,caring and down to earth…. I l...",1


In [24]:
# convert "age" column to integer
scammers_df.loc[:, "age"] = scammers_df["age"].astype(int)

In [25]:
# bin the ages value in scam dataframe
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
scammers_df['age_group'] = pd.cut(scammers_df['age'], bins=bins, labels=labels, right=False)
scammers_df['age_group'] = scammers_df['age_group'].astype('object')

In [26]:
scammers_df

,age,location,ethnicity,occupation,status,description,scam,age_group
0,54,"Washingtom DC, United States, or Kabul, Afghan...",native american,medical,widowed,"Am an easy going type of woman, very resourcef...",1,51-60
1,63,"Ohio, or Colorado, or Arkansas, United States",white,engineering,widowed,"I am independent man who has my own busy life,...",1,61-70
2,47,"Los Angleles, California, United States",native american,engineering,divorced,"Hey mates, I am Rob i am looking for a serious...",1,41-50
3,49,"Rockville, Maryland, United States, or Kabul, ...",white,government,widowed,"Hello,i am here to look for a good relationshi...",1,41-50
4,42,"London, United Kingdom, or Helsinki, Finland",white,self-employed,divorced,"a nice cool, gentle, honest and funning person...",1,41-50
...,...,...,...,...,...,...,...,...
3235,34,"Donetsk, Ukraine",white,sales,single,"I am – this is a simple soul, which difficult ...",1,31-40
3236,55,"Texas City, United States",white,contractor,single,looking to find love,1,51-60
3237,45,"Fort Lauderdale, Florida, United States",native american,military,single,Hello am Ron Riley Jack and am new here i am l...,1,41-50
3238,44,"London, United Kingdom",white,other,widowed,"I am Honest Man,caring and down to earth…. I l...",1,41-50


# Real profiles

In [ ]:
real_df = pd.read_csv('../data/real_profile.csv')
real_df.replace("-", np.nan, inplace=True)
real_df.head()

In [4]:
real_df = real_df.drop(["gender", "username", "children", "orientation", "religion", "smoking", "drinking", "intent", "match_age"], axis=1)

In [5]:
# extract just the age from the age column
real_df["age"] = real_df["age"].str.extract(r'(\d+)')

In [6]:
real_df

,age,location,status,ethnicity,occupation,description
0,33,"Caracas, Distrito Capital, Venezuela",single,white,Jefe,NaN
1,58,"San José, Costa Rica",separated,white,Administrativa,"Soy muy simple, fácil de llevar. No me gustan ..."
2,50,"Felton, DE 19943, USA",single,white,disabled,NaN
3,50,"Swansea, IL, USA",separated,white,NaN,NaN
4,69,"Delray Beach, FL, USA",divorced,hispanic,NaN,NaN
...,...,...,...,...,...,...
8269,49,"Maracaibo, Zulia, Venezuela",separated,hispanic,NaN,NaN
8270,32,"Zacatecas, Josefa Zozaya, Guadalupe, N.L., México",in relationship,hispanic,NaN,NaN
8271,39,"Cd Ojeda, Zulia, Venezuela",single,hispanic,Carpenter,I'm a carpenter and music lover looking for a ...
8272,53,"Chattanooga, TN, USA",single,white,Carpenter,NaN


In [ ]:
# bin the ages value in scam dataframe
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
real_df['age_group'] = pd.cut(real_df['age'], bins=bins, labels=labels, right=False)
real_df['age_group'] = real_df['age_group'].astype('object')

# Additional work to make the data more readable

Translating Descriptions to English 

In [ ]:
import time
import pandas as pd
from requests.exceptions import ReadTimeout

def translate_text(text, target_language='en'):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_language)
    return translated_text.text

def translate_chunk(chunk, target_language='en'):
    translated_chunk = chunk.apply(lambda x: translate_text(x, target_language))
    return translated_chunk

def translate_column(df, columns_to_translate, target_language='en', chunk_size=100, pause_duration=5, max_retries=3):
    for column_name in columns_to_translate:
        num_rows = len(df)
        for i in range(0, num_rows, chunk_size):
            retries = 0
            while retries < max_retries:
                try:
                    chunk = df.iloc[i:i+chunk_size][column_name]
                    translated_chunk = translate_chunk(chunk, target_language)
                    df.iloc[i:i+chunk_size, df.columns.get_loc(column_name)] = translated_chunk
                    print(f"Translated {len(translated_chunk)} rows. Pausing for {pause_duration} seconds...")
                    time.sleep(pause_duration)
                    break  # Exit the retry loop if translation succeeds
                except ReadTimeout:
                    print(f"Timeout error occurred. Retrying...")
                    retries += 1
            else:
                print(f"Max retries reached. Skipping chunk.")
                continue

    # Save the translated dataframe to a new CSV file
    # df.to_csv(output_csv_path, index=False)
    return df



# Scaling down the categories

Employment type

In [ ]:
def employ(level):
    level = level.lower()
    if any(x in level for x in ['army', 'milit', 'marine', 'soldier', 'captain', 'general', 'solda', 'force']):
        return "military"
    elif any(x in level for x in ['stud?ent', 'studi', 'coll']):
        return "student"
    elif any(x in level for x in ['self', 'own', 'independ', 'entre', 'freelanc', 'propia', 'autonomo']):
        return "self-employed"
    elif any(x in level for x in ['engin', 'ingenier', 'mechanic', 'mecanic', 'automot']):
        return "engineering"
    elif any(x in level for x in ['gover', 'civil', 'public', '^un[$ ]']):
        return "government"
    elif any(x in level for x in ['academ', 'profes', 'research', 'lectur', 'universi', 'ologist', 'phd']):
        return "academic"
    elif any(x in level for x in ['nurs', 'enfermer', 'care', 'trainer', 'nanny', 'baby', 'niñera', 'social']):
        return "carer"
    elif any(x in level for x in ['construc', 'carpent', 'roof', 'build', 'survey', 'ass?es', 'crane', 'equipment']):
        return "construction"
    elif any(x in level for x in ['secur', 'detect', 'polic', 'investig', 'guard', 'custod', 'correct']):
        return "security"
    elif any(x in level for x in ['econom', 'analy']):
        return "analyst"
    elif any(x in level for x in ['farm', 'agri']):
        return "agriculture"
    elif any(x in level for x in ['sail', 'sea', 'fish']):
        return "naval"
    elif any(x in level for x in ['weld', 'factory', 'manufact', 'machin', 'industr']):
        return "manufacturing"
    elif any(x in level for x in ['tech', 'inform', '^it[$ ]', 'telecom', 'téch', 'software', 'sistem', 'system', 'tecnico', 'técnico', 'program', 'network', 'comput', 'electro', 'teck', 'develop']):
        return "technology"
    elif any(x in level for x in ['retail', 'comerci', 'shop', 'clerk', 'store', 'wait', 'vend', 'sell', 'cashier', 'assist', 'tender', 'customer', 'asist', 'mesero', 'restaur', 'camarer']):
        return "service"
    elif any(x in level for x in ['tour', 'holiday', 'vacat', 'steward', 'flight', 'travel', 'turis', 'hotel']):
        return "tourism"
    elif any(x in level for x in ['sale', 'market', 'ventas']):
        return "sales"
    elif any(x in level for x in ['writ', 'journal', 'period']):
        return "writing"
    elif any(x in level for x in ['handy', 'repair', 'repare', 'maint', 'plumb', 'electr', 'manteni', 'hvac']):
        return "repair"
    elif 'estat' in level:
        return "real estate"
    elif any(x in level for x in ['teach', 'educa', 'docen', 'maestr', 'lehr']):
        return "teacher"
    elif any(x in level for x in ['manag', 'supervis']):
        return "manager"
    elif 'contra' in level:
        return "contractor"
    elif any(x in level for x in ['ama de casa', 'wife', 'mother', 'mom', 'home', 'hogar']):
        return "housewife"
    elif any(x in level for x in ['unemploy', 'desempl', 'not work']):
        return "unemployed"
    elif any(x in level for x in ['financ', 'bank', 'insur', 'trad', 'negoci', 'cajero']):
        return "finance"
    elif any(x in level for x in ['chef', 'cook', 'bake', 'co[cs]iner', 'hospitali', 'food']):
        return "hospitality"
    elif any(x in level for x in ['secret', 'admin', 'recep', 'office', 'human resources', 'clerical', 'profec', 'entry']):
        return "clerical"
    elif any(x in level for x in ['driver', 'transport', 'deliver', 'ship', 'chofer', 'pilot', 'logist', 'cargo']):
        return "transport"
    elif any(x in level for x in ['housekeep', 'clean', 'limpi', 'janitor']):
        return "cleaner"
    elif any(x in level for x in ['architec', 'arquitec']):
        return "architect"
    elif any(x in level for x in ['account', 'contad']):
        return "accounting"
    elif any(x in level for x in ['law', 'judge', 'solicitor', 'barrister', 'legal', 'attorney', 'abogad']):
        return "legal"
    elif any(x in level for x in ['music', 'sport', 'play', 'produc', 'músico', 'deporti', 'conduc', 'soccer']):
        return "entertainment"
    elif any(x in level for x in ['artist', 'art', 'paint', 'sculpt', 'boutique', 'photo', 'foto', 'choreo']):
        return "artist"
    elif any(x in level for x in ['jewel', 'antiq', 'print']):
        return "specialist"
    elif any(x in level for x in ['doctor', 'physic', 'ician', 'medic', 'psicolog', 'terap', 'therap', 'salud', 'health', 'médic', 'surgeon', 'denti', 'pharma']):
        return "medical"
    elif any(x in level for x in ['beaut', 'styl', 'estili', 'peluquer', 'hair', 'salon', 'manic']):
        return "beauty"
    elif any(x in level for x in ['fashion', 'model']):
        return "fashion"
    elif any(x in level for x in ['design', 'decor', 'flower', 'desiñ', 'deisñ']):
        return "designer"
    elif any(x in level for x in ['warehouse', 'work', 'opera', 'obrer', 'labor', 'labour', 'landscap', 'mining', 'mine', 'load', 'trabajo', 'pack', 'foreman']):
        return "manual"
    elif any(x in level for x in ['bus[iy]?nes', 'empresa', 'execut', 'direct', 'ceo', 'ejecut']):
        return "business"
    elif 'consult' in level:
        return "consultant"
    elif any(x in level for x in ['retir', 'jubilad', 'pension']):
        return "retired"
    elif 'disab' in level:
        return "disabled"
    elif pd.isnull(level):
        return None
    else:
        return "other"

Ethnic type

In [ ]:
def ethnise(level):
    if level.lower() not in ['asian', 'black', 'hispanic', 'middle eastern', 'mixed', 'native american', 'pacific islander', 'white']:
        return "other"
    else:
        return level